# Ingestion API

In [ ]:
import os
import pytest

from tiledb.vector_search.ingestion import ingest


config = {"vfs.s3.aws_access_key_id": os.getenv("AWS_ACCESS_KEY_ID"),
          "vfs.s3.aws_secret_access_key": os.getenv("AWS_SECRET_ACCESS_KEY")}

million = 1000000
ingest(array_uri="s3://tiledb-nikos/vector-search/test-ingestion-1",
   source_uri="s3://tiledb-nikos/vector-search/datasets/base.1B.u8bin",
   source_type="U8BIN",
   size=100 * million,
   training_sample_size=1 * million,
   config=config,
   verbose=True)

# Search API (flat)

In [1]:
import tiledb
import tiledb.vector_search as vs
import numpy as np


#db_uri = "s3://tiledb-andrew/sift/sift_base"
#ground_truth_uri = "s3://tiledb-andrew/sift/sift_groundtruth"

db_uri = "~/work/proj/vector-search/datasets/sift-andrew/sift_base"
ground_truth_uri = "~/work/proj/vector-search/datasets/sift-andrew/sift_groundtruth"
query_uri = "~/work/proj/vector-search/datasets/sift-andrew/sift_query"

n_vectors = 10

In [2]:
# load target query vectors
query_vectors = vs.load_as_array(query_uri)[:,:n_vectors]

# [ read matrix ~/work/proj/vector-search/datasets/sift-andrew/sift_query ]: 6 ms


In [3]:
print(type(query_vectors))
print(query_vectors.shape)
print(query_vectors)

<class 'numpy.ndarray'>
(128, 10)
[[ 1. 40. 28. ...  3.  1.  0.]
 [ 3. 25.  4. ...  1. 12. 11.]
 [11. 11.  3. ...  0.  4. 77.]
 ...
 [42.  3.  2. ... 25.  6. 53.]
 [48. 19. 54. ... 51. 17. 33.]
 [11. 13. 47. ... 15. 50.  2.]]


In [4]:
# load database
index = vs.FlatIndex(db_uri)

# [ read matrix ~/work/proj/vector-search/datasets/sift-andrew/sift_base ]: 185 ms


In [7]:
# run query against the query_vectors array with these parameters
result = index.query(query_vectors, k=10)

# [ Total time vq_query_heap ]: 744 ms


In [8]:
print(result)

[[932085 413247 669835 970797 340871 187470 402219 906750 376328 178811]
 [934876 413071 408764 125539 748397  67875 982379 618842 325865 177646]
 [561813 706838 408462  48044 748193 220473 652078 807599 323160 181997]
 [708177 880592 408855 191115 175336 460733 880346 569178 376277 181605]
 [706771 249062 551661 889039 716433 896005  59540 107468 323464 821938]
 [695756 400194 861882 546470 112586  27746 511412 207868 724549 807785]
 [435345 942339 406273 123985 107281 943689 595673 476215 176970 716433]
 [701258 880462 406324 546496 416604 669622 703786 762217 695175 358802]
 [455537 987636 551743 207768 371420 947299 982409 762237 803977 803100]
 [872728 941776 861530 831905 368246 515025 596714   5447 181435 866737]]


In [13]:
# load the ground truth array:
# this contains the vector IDs of the top neighbors for each vector in
# query_vectors

with tiledb.open(ground_truth_uri) as A:
    ground_truth = A[:10,:10]['a']

In [14]:
print(ground_truth)

[[932085 413247 669835 970797 340871 187470 402219 906750 376328 178811]
 [934876 413071 408764 125539 748397 220473 982379 618842 325865 177646]
 [561813 706838 408462  48044 748193  67875 652078 807599 323160 181997]
 [708177 880592 408855 191115 175336 460733 880346 569178 376277 181605]
 [706771 249062 551661 889039 716433 896005  59540 107468 323464 821938]
 [695756 400194 861882 546470 112586  27746 511412 207868 724549 807785]
 [435345 942339 406273 123985 107281 943689 595673 476215 176970 716433]
 [701258 880462 406324 546496 416604 669622 703786 762217 695175 358802]
 [455537 987636 551743 207768 371420 947299 982409 762237 803977 803100]
 [872728 941776 861530 831905 368246 515025 596714   5447 181435 866737]]


In [11]:
np.array(index).shape

()

In [12]:
with tiledb.open("~/work/proj/vector-search/datasets/sift-andrew/sift_base") as A:
                 print(A.schema)

ArraySchema(
  domain=Domain(*[
    Dim(name='rows', domain=(0, 127), tile=128, dtype='int32'),
    Dim(name='cols', domain=(0, 999999), tile=10000, dtype='int32'),
  ]),
  attrs=[
    Attr(name='a', dtype='float32', var=False, nullable=False),
  ],
  cell_order='col-major',
  tile_order='col-major',
  capacity=10000,
  sparse=False,
)

